In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tiff
import skimage.io as io
from skimage import color
import skimage.segmentation
from cellpose import models
import time
import pandas as pd

In [3]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


## DAPI Extraction

Use `DAPI_extraction.ipynb` to get only DAPI intensity signal from spectrally unmixed component data. The output directory from there should be `in_dir` in the code below.

## Nuclear Segmentation 

In [8]:
def Cellpose_segmentor(in_dir, save_dir_seg, measure_time_fpath=None):
    '''
    Produces label masks as nuclear segmentation output of Cellpose segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields. All images should be in .tiff format
    save_dir_seg: str
        the path to the directory where the segmented images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''
    
    times_df_dict = {'img': [], 'time_seconds': []}
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = io.imread(os.path.join(in_dir, file), as_gray=True)
            # choosing the 'nuclei' in-built model from CellPose
            model = models.Cellpose(model_type='nuclei')
            # specifying that the images are grayscaled
            channels = [0, 0]
            start_time = time.time()
            # predicting nuclei using the CellPose model
            label_mask, flows, styles, diams = model.eval(DAPI_gray, diameter=None, channels=channels)
            end_time = time.time()
            io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)
            time_taken = end_time - start_time
            times_df_dict['img'].append(file)
            times_df_dict['time_seconds'].append(time_taken)
        else:
            pass

    if measure_time_fpath is not None:
        times_df = pd.DataFrame(times_df_dict)
        times_df.to_csv(measure_time_fpath)
    else:
        pass
        
    return None

## Post Processing 
Post-processing of predicted label masks into binary masks for evaluation. The overall effect of this step is to separate touching nuclei by adding a 2-pixel gap (one-pixel erosion on each nucleus). Next, binary masks (both segmented and ground truth data) can be used to compute the F1-scores by utilizing connected component analysis. 

In [3]:
def postprocess(save_dir_seg, save_dir_bin):
    '''
    Produces binary masks for evaluation from the labelled mask output from the Cellpose algorithm

    Parameters
    save_dir_seg: str
        the path to the directory containing the label masks output by the Cellpose algorithm with the Cellpose_segmentor function. All images should
        be in .tiff format
    save_dir_bin: str
        the path to the directory where the binary images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    # iterating over all files in the input directory
    for file in os.listdir(save_dir_seg):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # Finding the pixels that are touching between any two nuclei
            label_mask = io.imread(os.path.join(save_dir_seg, file))
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir_bin, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

## Running the segmentation pipeline

In [9]:
# Replace the strings for in_dir, save_dir_seg and save_dir_bin to paths on the local machine
in_dir = r"D:\nuclear_seg\external_DAPI_tiff"
save_dir_seg = r"D:\nuclear_seg\Cellpose_preds\seg"
save_dir_bin = r"D:\nuclear_seg\Cellpose_preds\bin"
# To create binary masks using Cellpose segmentation algorithm
Cellpose_segmentor(in_dir, save_dir_seg, measure_time_fpath=r"D:\nuclear_seg\Cellpose_preds\Cellpose_times.csv")
postprocess(save_dir_seg,save_dir_bin)

C:\Users\Waluigi\AppData\Roaming\Python\Python310\site-packages\cellpose\resnet_torch.py:275: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename,

NameError: name 'pd' is not defined